# Convert v1 to v2

# Import packages

In [1]:
import torch
from pprint import pprint
from pathlib import Path

from chemprop.utils.v1_to_v2 import convert_model_dict_v1_to_v2
from chemprop.models.model import MPNN


# Change model paths here

In [2]:
chemprop_dir = Path.cwd().parent
model_v1_input_path =  chemprop_dir / "tests/data/example_model_v1_regression_mol.pt" # path to v1 model .pt file
model_v2_output_path = Path.cwd() / "converted_model.ckpt" # path to save the converted model .ckpt file

# Load v1 model .pt file

In [3]:
model_v1_dict = torch.load(model_v1_input_path)

In [4]:
# Here are all the keys that is stored in v1 model
pprint(list(model_v1_dict.keys()))

['args',
 'state_dict',
 'data_scaler',
 'features_scaler',
 'atom_descriptor_scaler',
 'bond_descriptor_scaler',
 'atom_bond_scaler']


In [5]:
# Here are the input arguments that is stored in v1 model
pprint(model_v1_dict['args'].__dict__)

{'activation': 'ReLU',
 'adding_bond_types': True,
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_constraints': [],
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'atom_targets': [],
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_constraints': [],
 'bond_descriptor_scaling': True,
 'bond_descriptors': None,
 'bond_descriptors_path': None,
 'bond_descriptors_size': 0,
 'bond_features_size': 0,
 'bond_targets': [],
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'constraints_path': None,
 'crossval_index_dir': None,
 'crossval_index_file': None,
 'crossval_index_sets': None,
 'cuda': False,
 'data_path': '/Users/hwpang/Software/chemprop/tests/data/regression.csv',
 'data_weights_path': Non

In [6]:
# Here are the state_dict that is stored in v1 model
pprint(list(model_v1_dict['state_dict'].keys()))

['encoder.encoder.0.cached_zero_vector',
 'encoder.encoder.0.W_i.weight',
 'encoder.encoder.0.W_h.weight',
 'encoder.encoder.0.W_o.weight',
 'encoder.encoder.0.W_o.bias',
 'readout.1.weight',
 'readout.1.bias',
 'readout.4.weight',
 'readout.4.bias']


# Convert loaded v1 model dictionary into v2 model dictionary

In [7]:
model_v2_dict = convert_model_dict_v1_to_v2(model_v1_dict)

In [8]:
# Here are all the keys in the converted model
pprint(list(model_v2_dict.keys()))

['epoch',
 'global_step',
 'pytorch-lightning_version',
 'state_dict',
 'loops',
 'callbacks',
 'optimizer_states',
 'lr_schedulers',
 'hparams_name',
 'hyper_parameters']


In [9]:
# Here are all the keys in the converted state_dict
pprint(list(model_v2_dict['state_dict'].keys()))

['w_t',
 'message_passing.W_i.weight',
 'message_passing.W_h.weight',
 'message_passing.W_o.weight',
 'message_passing.W_o.bias',
 'predictor.loc',
 'predictor.scale',
 'predictor.ffn.0.weight',
 'predictor.ffn.0.bias',
 'predictor.ffn.3.weight',
 'predictor.ffn.3.bias']


In [10]:
# Here are all the keys in the converted hyper_parameters
pprint(list(model_v2_dict['hyper_parameters'].keys()))

['batch_norm',
 'metrics',
 'w_t',
 'warmup_epochs',
 'init_lr',
 'max_lr',
 'final_lr',
 'message_passing',
 'agg',
 'predictor']


# Save

In [11]:
torch.save(model_v2_dict, model_v2_output_path)

# Load converted model

In [12]:
mpnn = MPNN.load_from_checkpoint(model_v2_output_path)

In [13]:
# now visually check the converted model is what is expected
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=147, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=433, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
  )
  (agg): MeanAggregation()
  (bn): Identity()
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.0, inplace=False)
      (3): Linear(in_features=300, out_features=1, bias=True)
    )
  )
)